In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dash
from dash import dcc,html
from dash.dependencies import Input,Output
import requests
import io
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
from dash import no_update

url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv"
response=requests.get(url)
if response.status_code==200:
    text=io.BytesIO(response.content)
    df=pd.read_csv(text)
    print("Data read into a pandas dataframe!")
else:
    print(f"Error downloading data: {response.status_code}")

df['Year']=pd.to_datetime(df['Date']).dt.year
df['Month']=pd.to_datetime(df['Date']).dt.month_name()


app=dash.Dash(__name__)
app.layout=html.Div([
                        html.H1('Australia Wildfire Dashboard',style={'font-size':40,'textAlign':'center','color':'red'}),
                        html.Div(['Select Region',dcc.RadioItems(options=[
                                                        {'label': 'New South Wales', 'value': 'NSW'},
                                                        {'label': 'Northern Territory', 'value': 'NT'},
                                                        {'label': 'Queensland', 'value': 'QL'},
                                                        {'label': 'South Australia', 'value': 'SA'},
                                                        {'label': 'Tasmania', 'value': 'TA'},
                                                        {'label': 'Victoria', 'value': 'VI'},
                                                        {'label': 'Western Australia', 'value': 'WA'}
    ],value='NSW',id='radio-items',labelStyle={'display': 'inline-block', 'margin': '10px'},
    inline=True,style={'font-size':10})],style={'font-size':20}),
                        html.Div(['Select Year',dcc.Dropdown(options=[
                                                                {'label': '2005', 'value': 2005},
                                                                {'label': '2006', 'value': 2006},
                                                                {'label': '2007', 'value': 2007},
                                                                {'label': '2008', 'value': 2008},
                                                                {'label': '2009', 'value': 2009},
                                                                {'label': '2010', 'value': 2010},
                                                                {'label': '2011', 'value': 2011},
                                                                {'label': '2012', 'value': 2012}]
                            ,value=2005,id='dropdown',clearable=False,searchable=False,placeholder='Select an option',style={'font-size':10})],style={'font-size':20}),
                        html.Br(),
                        html.Br(),
                        html.Div([
                            html.Div(dcc.Graph(id='pie_chart')),
                            html.Div(dcc.Graph(id='bar_chart'))]
                            ,style={'display':'flex'})
                        ])


@app.callback([
                Output(component_id='pie_chart',component_property='figure'),
                Output(component_id='bar_chart',component_property='figure')],
                [   Input(component_id='radio-items',component_property='value'),
                    Input(component_id='dropdown',component_property='value')]
                    )


def reg_year_display(input_region,input_year):  
    #data
   y_r_data = df[(df['Region'] == input_region)&(df['Year']==input_year)]
   #y_r_data = region_data[region_data['Year']==input_year]
    #Plot one - Monthly Average Estimated Fire Area   
   est_data = y_r_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
   fig1 = px.pie(est_data, values='Estimated_fire_area', names='Month', title="{} : Monthly Average Estimated Fire Area in year {}".format(input_region,input_year))   
     #Plot two - Monthly Average Count of Pixels for Presumed Vegetation Fires
   veg_data = y_r_data.groupby('Month')['Count'].mean().reset_index()
   fig2 = px.bar(veg_data, x='Month', y='Count', title='{} : Average Count of Pixels for Presumed Vegetation Fires in year {}'.format(input_region,input_year))    
   return [fig1,
            fig2 ]

if __name__=='__main__':
    app.run_server(port=8060)


Data read into a pandas dataframe!
